In [7]:
import Pcb_Car 
import time

car = Pcb_Car.Pcb_Car()

car.Ctrl_Servo(1, 90)  # S1 舵机转动到90度
time.sleep(0.5)

car.Ctrl_Servo(2, 90)  # S2 舵机转动到90度
time.sleep(0.5)

# car.Ctrl_Servo(1, 0)   # S1 舵机转动到0度
# time.sleep(0.5)

# car.Ctrl_Servo(2, 0)  # S2 舵机转动到0度
# time.sleep(0.5)

# car.Ctrl_Servo(1, 180)   # S1 舵机转动到180度
# time.sleep(0.5)

# car.Ctrl_Servo(2, 180)  # S2 舵机转动到180度
# time.sleep(0.5)

# car.Ctrl_Servo(1, 90)  # S1 舵机转动到90度
# time.sleep(0.5)

# car.Ctrl_Servo(2, 90)  # S2 舵机转动到90度
# time.sleep(0.5)

# car.Ctrl_Servo(1, 0)   # S1 舵机转动到0度
# time.sleep(0.5)

# car.Ctrl_Servo(2, 0)  # S2 舵机转动到0度
# time.sleep(0.5)

In [8]:
car.Car_Run(50, 50)  # 50 的速度前进 2秒， 速度取值范围 0~255
time.sleep(2)
car.Car_Stop()

In [7]:
car.Car_Back(50, 50)  # 50 的速度后退 2秒， 
time.sleep(2)
car.Car_Stop()

In [11]:
car.Car_Left(0, 50)  # 50 的速度左转 2秒， 
time.sleep(2)
car.Car_Stop()

In [16]:
car.Car_Right(50, 0)  # 50 的速度右转 2秒， 
time.sleep(2)
car.Car_Stop()

In [22]:
car.Car_Spin_Left(50, 50)  # 50 的速度左旋 2 秒
time.sleep(2)
car.Car_Stop()

In [21]:
car.Car_Spin_Right(50, 50)  # 50 的速度右旋 2 秒
time.sleep(2)
car.Car_Stop()

In [8]:
import Pcb_Car 
import time
import threading

# 初始化舵机控制
car = Pcb_Car.Pcb_Car()

# 定义舵机旋转函数，指定舵机号和旋转范围
def rotate_servo_smoothly(servo_id, start_angle, end_angle, step_angle=1, delay=0.1):  # 增加延迟
    if start_angle < end_angle:
        for angle in range(start_angle, end_angle + 1, step_angle):
            car.Ctrl_Servo(servo_id, angle)
            time.sleep(delay)
    else:
        for angle in range(start_angle, end_angle - 1, -step_angle):
            car.Ctrl_Servo(servo_id, angle)
            time.sleep(delay)
    
    # 缓慢匀速回到90度位置
    current_angle = end_angle if end_angle <= 180 else 180
    return_to_initial(servo_id, current_angle, 90, step_angle, delay)

# 定义复位函数
def return_to_initial(servo_id, current_angle, target_angle, step_angle=1, delay=0.1):  # 增加延迟
    if current_angle < target_angle:
        for angle in range(current_angle, target_angle + 1, step_angle):
            car.Ctrl_Servo(servo_id, angle)
            time.sleep(delay)
    else:
        for angle in range(current_angle, target_angle - 1, -step_angle):
            car.Ctrl_Servo(servo_id, angle)
            time.sleep(delay)

# 控制两个舵机分别在不同的线程中执行旋转
def demo_two_servos_with_threads():
    servo1_thread = threading.Thread(target=rotate_servo_smoothly, args=(1, 0, 180, 1, 0.1))  # 减少步进角度
    servo2_thread = threading.Thread(target=rotate_servo_smoothly, args=(2, 0, 180, 1, 0.1))  # 减少步进角度

    servo1_thread.start()
    servo2_thread.start()

    servo1_thread.join()
    servo2_thread.join()

# 执行演示
demo_two_servos_with_threads()

In [ ]:
import RPi.GPIO as GPIO
import time 
import Pcb_Car

car = Pcb_Car.Pcb_Car()

# 设置GPIO口为BOARD编码方式
GPIO.setmode(GPIO.BOARD)

# 忽略警告信息
GPIO.setwarnings(False)


EchoPIN = 18 
TrigPIN = 16

# 设置超声波模块引脚的模式
GPIO.setup(EchoPIN, GPIO.IN)
GPIO.setup(TrigPIN, GPIO.OUT)


# TrigPIN 输入至少10us 

def Distance():
    GPIO.output(TrigPIN, GPIO.LOW)
    time.sleep(0.000002)
    GPIO.output(TrigPIN, GPIO.HIGH)
    time.sleep(0.000015)
    GPIO.output(TrigPIN, GPIO.LOW)

    t3 = time.time()
    while not GPIO.input(EchoPIN):
        t4 = time.time()
        if (t4 - t3) > 0.03:
            return -1

    t1 = time.time()
    while GPIO.input(EchoPIN):
        t5 = time.time()
        if (t5-t1) > 0.03:
            return -1

    t2 = time.time()
    time.sleep(0.01)

    dist =  ((t2 - t1)*340 /2) * 100
    # print(f"distance >>>> {dist}")
    return dist

def Distance_test():
    num = 0 
    ultrasonic = []
    while num < 5:
        distance = Distance()
        while int(distance) == -1:
            distance = Distance()
            
        while int(distance) >= 500 or int(distance) == 0:
            distance = Distance()
            
        ultrasonic.append(distance)
        num = num +1

    distance = (ultrasonic[1] + ultrasonic[2] + ultrasonic[3]) / 3
    print(f"[Distance_test] >>> {distance}")
    return distance

def avoid():
    distance = Distance_test()
    if distance < 20:
        car.Car_Stop()
        time.sleep(0.5)
        car.Car_Spin_Right(60, 60)
        time.sleep(1)
    else:
        car.Car_Run(50, 50)

try:
    while True:
        avoid()
except KeyboardInterrupt:
    pass

car.Car_Stop()
del car
print("Ending")
GPIO.cleanup()

[Distance_test] >>> 17.726977666219074
[Distance_test] >>> 19.52521006266276
[Distance_test] >>> 17.087936401367188
[Distance_test] >>> 21.693627039591473
[Distance_test] >>> 20.28044064839681
[Distance_test] >>> 20.252068837483723
[Distance_test] >>> 19.410371780395508
[Distance_test] >>> 18.0512269337972
[Distance_test] >>> 44.08844312032064
[Distance_test] >>> 42.46854782104492
[Distance_test] >>> 42.74956385294596
[Distance_test] >>> 41.3823127746582
[Distance_test] >>> 63.596089680989586
[Distance_test] >>> 67.60191917419434
[Distance_test] >>> 76.73223813374837
[Distance_test] >>> 71.98333740234375
[Distance_test] >>> 72.17923800150554
[Distance_test] >>> 57.26917584737142
[Distance_test] >>> 7.757663726806641
[Distance_test] >>> 38.26006253560384
[Distance_test] >>> 37.05898920694987
[Distance_test] >>> 5.843242009480794
[Distance_test] >>> 80.97179730733235
[Distance_test] >>> 72.93987274169922
[Distance_test] >>> 66.44948323567708
[Distance_test] >>> 67.44384765625
[Distance_t

In [ ]:
import RPi.GPIO as GPIO
import time
import Pcb_Car

PIN = 36
buzzer = 32

GPIO.setmode(GPIO.BOARD)

GPIO.setwarnings(False)

ir_repeat_cnt = 0

car = Pcb_Car.Pcb_Car()

def init():
    GPIO.setup(PIN, GPIO.IN, GPIO.PUD_UP)
    GPIO.setup(buzzer, GPIO.OUT)

    print("IR test start...") 


def whistle():
    p = GPIO.PWM(buzzer, 400)
    p.start(50)
    time.sleep(0.5)
    p.stop()
    
    
    
def exec_cmd(key_val):
    if key_val==0x45:  # power
        car.Ctrl_Servo(1, 90)
        car.Ctrl_Servo(2, 90)
        car.Car_Stop()
    elif key_val == 0x40:  # button  +
        car.Car_Run(50, 50)  # move forward
    elif key_val == 0x15:  # BUTTON pause
        car.Car_Stop() 
        
    elif key_val == 0x07:  # button  << 
        car.Car_Left(0, 50) # left 
        
    elif key_val == 0x47:  # MENU button
        whistle()
        
    elif key_val == 0x09:  # button >>
        car.Car_Right(50, 0) 
        
    elif key_val == 0x16:  # button 0
        car.Car_Spin_Left(50, 50) 
        
    elif key_val == 0x19:  # button -
        car.Car_Back(50, 50) 
        
    elif key_val == 0x0d:  # button c
        car.Car_Spin_Right(50, 50)
        
    elif key_val == 0x0c:  # button 1
        car.Ctrl_Servo(1, 0) 
        
    elif key_val == 0x18:  # button 2
        car.Ctrl_Servo(1, 90) 
        
    elif key_val == 0x5e:  # button 3
        car.Ctrl_Servo(1, 180) 

    elif key_val == 0x08:  # button 4
        car.Ctrl_Servo(2, 0) 
        

    elif key_val == 0x1c:  # button 5
        car.Ctrl_Servo(2, 90) 

    elif key_val == 0x5a:  # button  6
        car.Ctrl_Servo(2, 180) 

    else:
        print(key_val)
        print("no cmd")

try:
    init()
    while True:
        # 等待红外起始低电平
        if GPIO.input(PIN) == 0:
            ir_repeat_cnt = 0
            count = 0
            while GPIO.input(PIN) == 0 and count < 200:      # 9 ms 低电平
                count += 1
                time.sleep(0.00006)

            count = 0
            while GPIO.input(PIN) == 1 and count < 80:       # 4.5 ms 高电平
                count += 1
                time.sleep(0.00006)

            idx = 0
            cnt = 0
            data = [0, 0, 0, 0]

            for i in range(32):                              # 32 bit 数据
                count = 0
                while GPIO.input(PIN) == 0 and count < 15:   # 560 µs 低电平
                    count += 1
                    time.sleep(0.00006)

                count = 0
                while GPIO.input(PIN) == 1 and count < 40:   # 判断是 0 还是 1
                    count += 1
                    time.sleep(0.00006)

                if count > 9:                                # 逻辑 1
                    data[idx] |= 1 << cnt

                if cnt == 7:                                 # 下一个字节
                    cnt = 0
                    idx += 1
                else:
                    cnt += 1

            # 校验和：低 8 位 + 高 8 位 == 0xFF
            if data[0] + data[1] == 0xFF and data[2] + data[3] == 0xFF:
                print("Get the key: 0x%02x" % data[2])
                exec_cmd(data[2])

        else:
            # 判断红外遥控接收器是否松开，但为复现瞬时时间约 11 ms，所以这里逻辑是读取 110 × 0.001
            if ir_repeat_cnt > 110:
                ir_repeat_cnt = 0
                car.Car_Stop()
            else:
                time.sleep(0.001)
                ir_repeat_cnt += 1

except KeyboardInterrupt:
    pass

print("Ending")
GPIO.cleanup()
